Link to dataset: [Here](https://www.kaggle.com/aaronschlegel/austin-animal-center-shelter-intakes-and-outcomes)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import helper as hlp
import seaborn as sns

# Pretty display for notebooks
%matplotlib inline

## Fetch dataset and create in & out dataframes

In [2]:
incomes_endpoint = 'https://data.austintexas.gov/resource/wter-evkm.json'
outcomes_endpoint = 'https://data.austintexas.gov/resource/9t4d-g238.json'

import requests
params = {'$limit': 10000000} # If not specified it only returns 1000 records
r = requests.get(incomes_endpoint, params)
in_df = pd.DataFrame(r.json())
r = requests.get(outcomes_endpoint, params)
out_df = pd.DataFrame(r.json())

in_df['income_time'] = pd.to_datetime(in_df['datetime'])
out_df['outcome_time'] = pd.to_datetime(out_df['datetime'])
out_df['date_of_birth'] = pd.to_datetime(out_df['date_of_birth'])

in_df.drop(['datetime','datetime2'], axis=1, inplace=True)
out_df.drop(['monthyear', 'datetime'], axis=1, inplace=True)

print('Incomes shape',in_df.shape)
print('Outcomes shape',out_df.shape)


Incomes shape (122993, 11)
Outcomes shape (122839, 11)


In [3]:
in_df.head(5)

,animal_id,found_location,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,color,name,income_time
0,A827590,4812 E Riverside Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Labrador Retriever Mix,Brown/Brown,NaN,2020-12-26 16:27:00
1,A827289,12433 Dessau Rd in Austin (TX),Owner Surrender,Normal,Cat,Spayed Female,3 years,Domestic Shorthair,Blue/White,Essie,2020-12-26 15:52:00
2,A827550,1500 E Riverside Dr in Austin (TX),Public Assist,Normal,Dog,Neutered Male,2 years,Unknown,Black,Tigga,2020-12-26 15:34:00
3,A827584,Montopolis Dr And Ponca St in Austin (TX),Stray,Normal,Dog,Intact Male,2 years,Miniature Poodle,White,NaN,2020-12-26 14:26:00
4,A554278,3002 Birdwood Circle in Austin (TX),Stray,Normal,Dog,Neutered Male,13 years,Dachshund Mix,Brown/White,Oscar,2020-12-26 14:20:00


In [4]:
out_df.head(5)

,animal_id,name,date_of_birth,outcome_type,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,outcome_subtype,outcome_time
0,A821019,Spot,2017-04-03,Adoption,Dog,Neutered Male,3 years,Pit Bull,White/Black,NaN,2020-12-08 12:37:00
1,A824438,*Rose,2011-11-27,Adoption,Dog,Spayed Female,9 years,German Shepherd,Tan/Black,NaN,2020-12-08 12:27:00
2,A825587,*Ludwig,2011-11-06,Adoption,Cat,Neutered Male,9 years,Domestic Medium Hair,Cream Tabby,Foster,2020-12-08 12:22:00
3,A819626,NaN,2020-06-25,Adoption,Cat,Neutered Male,5 months,Domestic Shorthair,White/Black,Foster,2020-12-08 11:53:00
4,A819624,NaN,2020-06-25,Adoption,Cat,Neutered Male,5 months,Domestic Shorthair,Black,Foster,2020-12-08 11:52:00


## Create unique id to merge in & out datasets

In [5]:
in_df['animal_id'].value_counts()

A721033    33
A718223    14
A718877    12
A706536    11
A717053     9
           ..
A818640     1
A694907     1
A679670     1
A756627     1
A814985     1
Name: animal_id, Length: 109957, dtype: int64

In [6]:
# In order to merge the two datasets we need a unique parameter to link the datasets. animal_id can't be used 
# because one animal may stay in the shelter more than once. Thus, the a new id must be created.

In [7]:
def create_income_id(df, dataframe_type):
    
    # Get for each animal_id its occurrence number in the shelter
    occurrence = df.groupby(['animal_id'])[dataframe_type + '_time'].rank(method='dense', ascending=False).astype(int)
    
    # The new id will be df['animal_id'] + '_' + rank where rank is the number of occurence of the animal in the shelter 
    df['income_id'] = df['animal_id'] + '_' + occurrence.astype(str)
    return df

In [8]:
in_df = create_income_id(in_df, 'income')
out_df = create_income_id(out_df, 'outcome')

## Check columns in common

In [9]:
print('Columns of the income ->',in_df.columns)
print('Columns of the ou0tcome ->',out_df.columns)
set(in_df.columns).intersection(out_df.columns)

Columns of the income -> Index(['animal_id', 'found_location', 'intake_type', 'intake_condition',
       'animal_type', 'sex_upon_intake', 'age_upon_intake', 'breed', 'color',
       'name', 'income_time', 'income_id'],
      dtype='object')
Columns of the ou0tcome -> Index(['animal_id', 'name', 'date_of_birth', 'outcome_type', 'animal_type',
       'sex_upon_outcome', 'age_upon_outcome', 'breed', 'color',
       'outcome_subtype', 'outcome_time', 'income_id'],
      dtype='object')


{'animal_id', 'animal_type', 'breed', 'color', 'income_id', 'name'}

## Remove columns to avoid duplicates in merge

In [10]:
#in_df.drop(['index'], axis=1, inplace=True)
out_df.drop(['animal_type', 'breed', 'color','name', 'animal_id'], axis=1, inplace=True)

#### Merge

In [11]:
in_df.set_index('income_id', inplace=True)
out_df.set_index('income_id', inplace=True)
# Merge datasets using type 'inner' to only take animals with both income and outcome 
in_out_df = pd.merge(out_df, in_df, how='inner', 
                  right_index=True, left_index=True, suffixes=['_outcome', '_income'])

In [12]:
print(in_out_df.columns)
print('Income shape', in_df.shape)
print('Outcome shape', out_df.shape)
print('In/out shape', in_out_df.shape)
in_out_df.head()
# TODO: see why the diference in row numbers (animals with income but not yet an outcome?) 

Index(['date_of_birth', 'outcome_type', 'sex_upon_outcome', 'age_upon_outcome',
       'outcome_subtype', 'outcome_time', 'animal_id', 'found_location',
       'intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake',
       'age_upon_intake', 'breed', 'color', 'name', 'income_time'],
      dtype='object')
Income shape (122993, 11)
Outcome shape (122839, 6)
In/out shape (121834, 17)


,date_of_birth,outcome_type,sex_upon_outcome,age_upon_outcome,outcome_subtype,outcome_time,animal_id,found_location,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,color,name,income_time
income_id,,,,,,,,,,,,,,,,,
A006100_1,2007-07-09,Return to Owner,Neutered Male,10 years,NaN,2017-12-07 00:00:00,A006100,Colony Creek And Hunters Trace in Austin (TX),Stray,Normal,Dog,Neutered Male,10 years,Spinone Italiano Mix,Yellow/White,Scamp,2017-12-07 14:07:00
A006100_2,2007-07-09,Return to Owner,Neutered Male,7 years,NaN,2014-12-20 16:35:00,A006100,8700 Research Blvd in Austin (TX),Public Assist,Normal,Dog,Neutered Male,7 years,Spinone Italiano Mix,Yellow/White,Scamp,2014-12-19 10:21:00
A006100_3,2007-07-09,Return to Owner,Neutered Male,6 years,NaN,2014-03-08 17:10:00,A006100,8700 Research in Austin (TX),Public Assist,Normal,Dog,Neutered Male,6 years,Spinone Italiano Mix,Yellow/White,Scamp,2014-03-07 14:26:00
A047759_1,2004-04-02,Transfer,Neutered Male,10 years,Partner,2014-04-07 15:12:00,A047759,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,10 years,Dachshund,Tricolor,Oreo,2014-04-02 15:55:00
A134067_1,1997-10-16,Return to Owner,Neutered Male,16 years,NaN,2013-11-16 11:54:00,A134067,12034 Research Blvd in Austin (TX),Public Assist,Injured,Dog,Neutered Male,16 years,Shetland Sheepdog,Brown/White,Bandit,2013-11-16 09:02:00


### Data cleaning

In [13]:
in_out_df['animal_type'].value_counts()

Dog          69051
Cat          45693
Other         6498
Bird           573
Livestock       19
Name: animal_type, dtype: int64

In [14]:
# Remove animals of type 'Livestock' and 'Other' because its of no use for the recomendation system
in_out_df = in_out_df.loc[in_out_df['animal_type'] != 'Other']
in_out_df = in_out_df.loc[in_out_df['animal_type'] != 'Livestock']

In [20]:
# See the different outcome types
print(in_out_df['outcome_type'].unique())
print(in_out_df.shape[0])
print(in_out_df['outcome_type'].value_counts())

['Transfer' 'Relocate' 'Adoption' 'Euthanasia' 'Died' 'Rto-Adopt'
 'Missing' 'Disposal']
93520
Adoption      53094
Transfer      34869
Euthanasia     3758
Died            930
Rto-Adopt       667
Disposal        129
Missing          63
Relocate         10
Name: outcome_type, dtype: int64


In [19]:
# Remove 'outcome_type' of 'Return to Owner'. For the recomendation system we don't want cases of 
# lost animals to train the model
in_out_df = in_out_df.loc[in_out_df['outcome_type'] != 'Return to Owner']

## Create column time in shelter using outcome and income times

In [17]:
in_out_df['time_in_shelter'] = in_out_df['outcome_time'] - in_out_df['income_time']
in_out_df['time_in_shelter'].describe()

count                      94126
mean     21 days 17:36:01.328432
std      52 days 15:56:06.979541
min         -2411 days +06:11:00
25%              3 days 19:41:00
50%              6 days 08:17:30
75%             23 days 00:08:45
max           1521 days 23:31:00
Name: time_in_shelter, dtype: object

In [18]:
# Remove rows where income_time is greater than outcome_time
in_out_df = in_out_df.loc[in_out_df['income_time'] < in_out_df['outcome_time']]

# Export dataframe

In [22]:
in_out_df.to_csv('./data/in_out_shelter.csv', encoding='utf-8')